# Information Retrieval and Web Analytics: Indexing + Modeling (TF-IDF) 

In [1]:
# if you do not have nltk the following command should work "python -m pip install nltk" 
import nltk
nltk.download('stopwords');

[nltk_data] Downloading package stopwords to /home/waze/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import time
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la
import json
import re
import csv
import sys

from gensim.summarization.bm25 import BM25
import pandas as pd
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_string
import nltk
nltk.download('punkt') # used in sent_tokenize

data_path = './data/one2.json'

[nltk_data] Downloading package punkt to /home/waze/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Load data into memory

As mentioned above the dataset is stored in a tsv file ```parsed_input_500.tsv```and it contains 500 Wikipedia article (one article per line). For each article we have id, title and body separated by "|".

In [3]:
docs_path = data_path
with open(docs_path) as fp:
    lines = fp.readlines()
tweets = [l.strip().replace(' +', ' ') for l in lines]

In [4]:
print("Total numer of tweets in the corpus: {}" .format(len(tweets)))

Total numer of tweets in the corpus: 1001


In [5]:
def cleanTweet(tweetText):
    """
    Preprocess the article text (title + body) removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """
        
    stemming = PorterStemmer()
    stops = set(stopwords.words("english"))
    ## START CODE
    line= tweetText.lower() ## Transform in lowercase
    line=re.sub('[:\[\]&%$\"\'!./,;:?=¿^\-#_*+)<>(¡@]','',line)
    line= line.split() ## Tokenize the text to get a list of terms
    line= [word for word in line if word not in stops]  ##eliminate the stopwords (HINT: use List Comprehension)
    line= [stemming.stem(word) for word in line] ## perform stemming (HINT: use List Comprehension)
    ## END CODE
    return line
    

In [6]:
def getRelevantInfo(tweet):
    dictRelevantInfo ={}
    data = json.loads(tweet)
    hashtags = []
    urlsList = []
    text = ''
    date = data['created_at'] ## ??? RT o no RT
    try:
        isRt=True
        isRetweet=data["retweeted_status"]
        idTweet=isRetweet["id_str"]
        text = isRetweet['text']
        username = isRetweet['user']['screen_name']
        urls = isRetweet['entities']['urls']
        rt_count = isRetweet['retweet_count']
        likes = isRetweet['favorite_count']
        
        for h in isRetweet['entities']['hashtags']:
            hashtags.append(h['text'])
        for url in urls:
            urlsList.append(url['url'])
            
    except:
        isRt=False
        idTweet=data["id_str"]
        text = data['text']
        username = data['user']['screen_name']
        urls = data['entities']['urls']
        rt_count=data['retweet_count']
        likes = data['favorite_count']
        
        for h in data['entities']['hashtags']:
            hashtags.append(h['text'])
            
        for url in urls:
            urlsList.append(url['url'])        
            
    dictRelevantInfo['tweetID'] = idTweet
    dictRelevantInfo['text'] = text
    dictRelevantInfo['tokens'] = cleanTweet(text)
    dictRelevantInfo['username'] = username
    dictRelevantInfo['date'] = date
    dictRelevantInfo['hashtags'] = hashtags
    dictRelevantInfo['likes'] = likes
    dictRelevantInfo['rt_count'] = rt_count
    dictRelevantInfo['urlsList'] = urlsList
    dictRelevantInfo['isRetweeted'] = isRt
    return dictRelevantInfo

In [7]:
cleanTweets = {}
for t in tweets:
    currentTweet=getRelevantInfo(t)
    tweetID=currentTweet['tweetID']
    isRtCurrent=currentTweet['isRetweeted']
    # Orignial tweet found, overwrite if retweet exist.
    if isRtCurrent == False:
        cleanTweets[tweetID] = currentTweet
    else:
        if tweetID in cleanTweets:
            continue
        else:
            cleanTweets[tweetID] = currentTweet

In [8]:
print("Length of cleaned tweets: ",len(cleanTweets))

Length of cleaned tweets:  709


In [9]:
def create_index(lines):
    """
    Impleent the inverted index
    
    Argument:
    lines -- collection of Wikipedia articles
    
    Returns:
    index - the inverted index (implemented through a python dictionary) containing terms as keys and the corresponding 
    list of document these keys appears in (and the positions) as values.
    """
    index=defaultdict(list) 
    tweetIndex = {} # dictionary to map tweets to page ids
    for line in lines.values(): # Remember, lines contain all tweets, each line is a tweet
        tweetID = line['tweetID']
        terms = line['tokens'] #page_title + page_text
        tweetIndex[tweetID]=line['text']  ## we do not need to apply get terms to title because it used only to print titles and not in the index
        
        ## ===============================================================        
        ## create the index for the current doc and store it in termdictPage
        ## termdictPage ==> { ‘term1’: [currentdoc, [list of positions]], ...,‘termn’: [currentdoc, [list of positions]]}
        
        ## Example: if the curr_doc has id 1 and his text is 
        ## "web retrieval information retrieval":
        
        ## termdictPage ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}
        
        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================
        
        termdictPage={}

        for position, term in enumerate(terms): # terms contains page_title + page_text. Loop over all terms
            try:
                # if the term is already in the index for the current page (termdictPage)
                # append the position to the corrisponding list
                
        ## START CODE
                termdictPage[term][1].append(position)  
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                termdictPage[term]=[tweetID, array('I',[position])] #'I' indicates unsigned int (int in python)
            
        #merge the current page index with the main index
        for termpage, postingpage in termdictPage.items():
            index[termpage].append(postingpage)
        
        ## END CODE                    
                    
    return index, tweetIndex

In [10]:
start_time = time.time()

# index is the dict of , index by twee ?¿?¿
# tweetIndex is the dict of tweets, indexed by tweetID
index, tweetIndex = create_index(cleanTweets)

print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time,2)))

Total time to create the index: 0.01 seconds


Notice that if you look in the index for ```researcher```you will not find any result, while if you look for ```research``` you will get some results. That happens because we are storing in the index stemmed terms.

In [11]:
#print("Index results for the term 'biden': {}\n".format(index['biden']))
print("Length of Index results for the term 'biden': {}\n".format(len(index['obama'])))
print("First 10 Index results for the term 'biden': \n{}".format(index['biden'][:10]))

Length of Index results for the term 'biden': 8

First 10 Index results for the term 'biden': 
[['1331952640574459905', array('I', [1])], ['1333092424936284163', array('I', [3])], ['1333092425611554823', array('I', [8])], ['1333088877343608832', array('I', [6])], ['1333026911799439361', array('I', [7])], ['1332712274000293889', array('I', [3])], ['1333092426953658368', array('I', [7])], ['1333023981646057473', array('I', [6])], ['1333087495509012480', array('I', [0])], ['1333088498975436804', array('I', [2])]]


In [12]:
def getResultsFromIndex(word, index):
    stemming = PorterStemmer()
    newWord=word.lower() ## Transform in lowercase
    newWord=re.sub('[:\[\]&%$\"\'!./,;:?=¿^\-#_*+)<>(¡@]','',word)
    newWord= stemming.stem(word) ## perform stemming
    print("Length of Index results for the term '{}': {}".format(newWord, len(index[newWord])))
    print("First 10 Index results for the term '{}': \n{}\n\n".format(newWord, index[newWord][:10]))

In [13]:
getResultsFromIndex("trump", index)
getResultsFromIndex("biden", index)
getResultsFromIndex("joe", index)

Length of Index results for the term 'trump': 326
First 10 Index results for the term 'trump': 
[['1332994175281868801', array('I', [6])], ['1333068258862370819', array('I', [0])], ['1333068652665516036', array('I', [3])], ['1332927520270913536', array('I', [8])], ['1333057200802107393', array('I', [9])], ['1333091717579501569', array('I', [2])], ['1333071374081011715', array('I', [5])], ['1333023981646057473', array('I', [3])], ['1333070773758685194', array('I', [0])], ['1332844785334358016', array('I', [1])]]


Length of Index results for the term 'biden': 155
First 10 Index results for the term 'biden': 
[['1331952640574459905', array('I', [1])], ['1333092424936284163', array('I', [3])], ['1333092425611554823', array('I', [8])], ['1333088877343608832', array('I', [6])], ['1333026911799439361', array('I', [7])], ['1332712274000293889', array('I', [3])], ['1333092426953658368', array('I', [7])], ['1333023981646057473', array('I', [6])], ['1333087495509012480', array('I', [0])], ['1333

### Querying the Index

Even if before we mentioned that in case of phrase queries we need to take into account the position of the terms in the document and we have implemented an index that would allow us to also work with this type of queries, here you are going to implement a search function that will query the index without take into account the trems' positions.


We will use english Free Text Queries, that means that the query we will query the index using  a sequence of english words as query, and the output will be the list of documents that contain any of the query terms. 

For instance if we write the query **"computer science"** the output will be the union of all documents containing the term "computer" with all documents containing the term "science".

In [14]:
def search(query, index):
    '''
    The output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    '''
    query=cleanTweet(query)
    tweets=set()
    for term in query:
    ## START DODE
        try:
            # store in termTweets the ids of the tweets that contain "term"                        
            termTweets=[posting[0] for posting in index[term]]
            # tweets = tweets Union termTweets
            tweets = tweets.union(termTweets)
        except:
            #term is not in index
            pass
    tweets=list(tweets)
    return tweets

In [15]:
print("Insert your query:\n")
query = input()
docs = search(query, index)    
top = 10

print("\n======================\nSample of {} results out of {} for the seached query:\n".format(top, len(docs)))
for d_id in docs[:top] :
    print("tweet_id= {}\ntweet_text: {}\n\n\n".format(d_id, tweetIndex[d_id]))

Insert your query:

joe biden

Sample of 10 results out of 159 for the seached query:

tweet_id= 1333088083802279938
tweet_text: Notice how Trump never makes any mention of Joe Biden’s full name let alone his middle name (Robinette) but makes s… https://t.co/m2DW6BjaJq



tweet_id= 1333092491785097216
tweet_text: @gtconway3d @realDonaldTrump I am grateful we are having this conversation post Biden win. I shudder even to imagin… https://t.co/tSK2WvRU2q



tweet_id= 1332816778578169861
tweet_text: With the mountains of evidence that keep piling up, believing that Joe Biden fairly won 80 million votes is more ou… https://t.co/0gQaQYOVxL



tweet_id= 1332754451183054848
tweet_text: Biden is too demented to cheat.

This was too well orchestrated #Demscheat



tweet_id= 1333054449405845504
tweet_text: Just three more wins in Pennsylvania, the swimsuit competition and a pie eating contest and then it’s PRESIDENT-ELECT Joe Biden 🤩🤩🤩



tweet_id= 1332843918455521280
tweet_text: Big congrats to 

### Ranking tf-idf

When searching in a search engine, we are interested in obtain the results sorted by relevance or by some other criteria. Notice that **the above results are not ranked**.

Here you are going to implement **tf-idf (Term Frequency — Inverse Document Frequency)** and use it to obtain a list of ordered results.

Tf-idf is a weighting scheme that assigns each term in a document a weight based on its term frequency (tf) and inverse document frequency (idf).  The higher the scores, more important the term is. 

##### TF
**tf** refers to the frequency of a term $t$ in a specific document $d$. The basic idea is that as a term appears more in the document it becomes more important. On the other side, if we only use pure term counts, longer documents will be favored more. Consider two documents with exactly the same content but one being twice longer by concatenating with itself.  The tf weights of each word in the longer document will be twice the shorter one, although they essentially have the same content. To deal with this issue we need to **normalize the term frequencies**.

$$tf_{t,d} = \dfrac{N_{t,d}}{||D||}\tag{1}$$



where ||D|| is the Euclidean norm. 


Let $D=[t_1, t_2, \dots, t_n]$ be the document vector where $t_i$ represent the frequency of the term $i$, the  Euclidean Norm is calculated as

$$\sqrt{\sum_{t=1}^{n}t_i{^2}}\tag{2}$$

Note that $||D||$ is the same for all terms of a document.


##### IDF
A drawback of tf is that it considers all terms equally important. However, less common terms are more discriminative than others. To deal with this issue we introduce **idf (inverse document frequency)** that takes into account the number of documents containing the term.

$$idf_t = log\dfrac{N}{df_t}\tag{3}$$

where:

- $N$ is the total number of documents;
- $df_t$ is the number of ocuments containing the term $t$.

The log operation is applied to avoid that terms that appears in a high number of documents are considered to be too much less important, in this way we are smoothing (dampening) this difference.


In [16]:
def create_index_tfidf(lines, numDocuments):
    """
    Implement the inverted index and compute tf, df and idf
    
    Argument:
    lines -- collection of Wikipedia articles
    numDocuments -- total number of documents
    
    Returns:
    index - the inverted index (implemented through a python dictionary) containing terms as keys and the corresponding 
    list of document these keys appears in (and the positions) as values.
    tf - normalized term frequency for each term in each document
    df - number of documents each term appear in
    idf - inverse document frequency of each term
    """
        
    index=defaultdict(list)
    tf=defaultdict(list) #term frequencies of terms in documents (documents in the same order as in the main index)
    df=defaultdict(int)         #document frequencies of terms in the corpus
    titleIndex=defaultdict(str)
    idf=defaultdict(float)
    
    for line in lines.values(): # Remember, lines contain all tweets, each line is a tweet
        #tweetID = line['tweetID']
        tweetID = line['tweetID']        
        terms = line['tokens']
        tweetIndex[tweetID]=line['text']           
        
        ## ===============================================================        
        ## create the index for the **current page** and store it in termdictPage
        ## termdictPage ==> { ‘term1’: [currentdoc, [list of positions]], ...,‘termn’: [currentdoc, [list of positions]]}
        
        ## Example: if the curr_doc has id 1 and his text is 
        ## "web retrieval information retrieval":
        
        ## termdictPage ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}
        
        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

        termdictPage={}

        for position, term in enumerate(terms): ## terms contains page_title + page_text
            try:
                # if the term is already in the dict append the position to the corrisponding list
                termdictPage[term][tweetID].append(position) 
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                termdictPage[term]=[tweetID, array('I',[position])] #'I' indicates unsigned int (int in python)
        
        #normalize term frequencies
        # Compute the denominator to normalize term frequencies (formula 2 above)
        # norm is the same for all terms of a document.
        norm=0
        for term, posting in termdictPage.items(): 
            # posting is a list containing doc_id and the list of positions for current term in current document: 
            # posting ==> [currentdoc, [list of positions]] 
            # you can use it to inferr the frequency of current term.
            norm+=len(posting[1])**2
        norm=math.sqrt(norm)


        #calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in termdictPage.items():     
            # append the tf for current term (tf = term frequency in current doc/norm)
            tf[term].append(np.round(len(posting[1])/norm,4))  ## SEE formula (1) above
            #increment the document frequency of current term (number of documents containing the current term)
            df[term]= len(termdictPage[term])  # increment df for current term
        
        #merge the current page index with the main index
        for termpage, postingpage in termdictPage.items():
            index[termpage].append(postingpage)
            
        # Compute idf following the formula (3) above. HINT: use np.log
        for term in df:
            idf[term] = np.round(np.log(float(numDocuments/df[term])),4)
            
    return index, tf, df, idf, tweetIndex


In [17]:
def createLikeRTIndex(lines, numDocuments):
    maxLikes = 0
    maxRT = 1
    for tweet in lines:
        if lines[tweet]['likes'] > maxLikes:
            maxLikes = lines[tweet]['likes']
        if lines[tweet]['rt_count'] > maxRT:
            maxRT = lines[tweet]['rt_count']
    for tweet in lines:
        lines[tweet]['likes_score'] = (-np.exp(-(lines[tweet]['likes']/50000))+1)
        lines[tweet]['rt_score'] = (-np.exp(-(lines[tweet]['rt_count']/25000))+1)

In [18]:
start_time = time.time()
numTweets = len(cleanTweets)
index, tf, df, idf, tweetIndex = create_index_tfidf(cleanTweets, numTweets)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time,2)))

Total time to create the index: 10.68 seconds


In [19]:
createLikeRTIndex(cleanTweets, numTweets)

print(cleanTweets['1323630540977852416']['likes'])
print(cleanTweets['1323630540977852416']['likes_score'])

57724
0.6847785905742336


In [20]:
def saveDictToCSV(d, filename):
    w = csv.writer(open(filename, "w"))
    for key, val in d.items():
        w.writerow([key, val])
        
def loadDictFromCSV(filename):
    tmpDict={}
    with open(filename,'r') as data: 
        for line in csv.reader(data): 
            tmpDict[line[0]]=line[1]
    return tmpDict

In [21]:
def rankDocuments(terms, docs, index, idf, tf, titleIndex):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    titleIndex -- mapping between page id and page title
    
    Returns:
    Print the list of ranked documents
    """
        
    # I'm interested only on the element of the docVector corresponding to the query terms 
    # The remaing elements would became 0 when multiplied to the queryVector
    docVectors=defaultdict(lambda: [0]*len(terms)) # I call docVectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary 
    queryVector=[0]*len(terms)    

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms) # get the frequency of each term in the query. 
    # Example: collections.Counter(["hello","hello","world"]) --> Counter({'hello': 2, 'world': 1})
    # HINT: use when computing tf for queryVector
    
    query_norm = la.norm(list(query_terms_count.values()))
    
    for termIndex, term in enumerate(terms): #termIndex is the index of the term in the query
        if term not in index:
            continue
                    
        ## Compute tf*idf(normalize tf as done with documents)
        queryVector[termIndex]=query_terms_count[term]/query_norm * idf[term]

        # Generate docVectors for matching docs
        for docIndex, (doc, postings) in enumerate(index[term]):
            # Example of [docIndex, (doc, postings)]
            # 0 (26, array('I', [1, 4, 12, 15, 22, 28, 32, 43, 51, 68, 333, 337]))
            # 1 (33, array('I', [26, 33, 57, 71, 87, 104, 109]))
            # term is in doc 26 in positions 1,4, .....
            # term is in doc 33 in positions 26,33, .....
            
            #tf[term][0] will contain the tf of the term "term" in the doc 26            
            if doc in docs:
                docVectors[doc][termIndex]=tf[term][docIndex] * idf[term]  # TODO: check if multiply for idf

    # calculate the score of each tweet
    # compute the cosine similarity between queyVector and each docVector:
    # HINT: you can use the dot product because in case of normalized vectors it corresponds to the cosine siilarity
    # see np.dot
    
    docScores=[ [(np.dot(curDocVec, queryVector)/(np.linalg.norm(curDocVec)*np.linalg.norm(queryVector)))*0.6+cleanTweets[doc]['likes_score']*0.2+cleanTweets[doc]['rt_score']*0.2, doc] for doc, curDocVec in docVectors.items() ]
    docScores.sort(reverse=True)
    resultDocs=[(x[0], x[1]) for x in docScores]
    #print document titles instead if document id's
    #resultDocs=[ titleIndex[x] for x in resultDocs ]
    if len(resultDocs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)    
    #print ('\n'.join(resultDocs), '\n')
    return resultDocs

In [22]:
def search_tf_idf(query, index):
    '''
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    '''
    query=cleanTweet(query)
    docs=set()
    for term in query:
        try:
            # store in termDocs the ids of the docs that contain "term"                        
            termDocs=[posting[0] for posting in index[term]]
            
            # docs = docs Union termDocs
            docs = docs.union(termDocs)
        except:
            #term is not in index
            pass
    docs=list(docs)
    ranked_docs = rankDocuments(query, docs, index, idf, tf, tweetIndex)   
    return ranked_docs

In [23]:
print("Insert your query:\n")
query = input()
ranked_docs = search_tf_idf(query, index)    
top = 20

print("\n======================\nTop {} results out of {} for the seached query:\n".format(top, len(ranked_docs)))
for d_score, d_id in ranked_docs[:top] :
    print("Tweet_id= {}\nTweet: {}\nLikes: {}\nRT: {}\nScore: {}\n\n".format(d_id, tweetIndex[d_id], cleanTweets[d_id]['likes'], cleanTweets[d_id]['rt_count'], d_score))

Insert your query:

joe biden

Top 20 results out of 159 for the seached query:

Tweet_id= 930065838387863552
Tweet: It's time to talk about former Vice President Joe Biden, the open sexual predator. A thread/moment...
Likes: 65597
RT: 49910
Score: 0.9189759037646573


Tweet_id= 1332721647741243392
Tweet: Actually, yes.
In 2012, Obama ran against a good, decent man. They differed on policy.
Joe Biden ran against the hu… https://t.co/yKxlgiPnsh
Likes: 83343
RT: 9319
Score: 0.8244660754495378


Tweet_id= 1332407714304110597
Tweet: Biden did poorly in big cities (Politico), except those of Detroit (more votes than people!), Philadelphia, Atlanta… https://t.co/0cCyv36E31
Likes: 255510
RT: 56691
Score: 0.8023458365947197


Tweet_id= 1332155514386599937
Tweet: A must read. Impossible for Biden to have overcome these, and even greater, odds! https://t.co/cmYFY0va6p
Likes: 191740
RT: 53268
Score: 0.7961925845187292


Tweet_id= 1332400909536653312
Tweet: Raise your hand if you're one of the 80,

In [24]:
#print(tf)
#print(dict(sorted(tf.items(), reverse=True, key=lambda item: item[1])))
print(dict(sorted(idf.items(), reverse=True, key=lambda item: item[1])))
# LOCURA (-e^-(x/50000)+1)

{'gop': 5.8707, 'amp': 5.8707, 'gopchairwoman': 5.8707, 'take': 5.8707, 'immedi': 5.8707, 'action': 5.8707, 'expos': 5.8707, 'fraud': 5.8707, 'correct': 5.8707, '113': 5.8707, 'elect': 5.8707, 'republican': 5.8707, 'parti': 5.8707, 'wil…': 5.8707, 'httpstcodwblj4kd8x': 5.8707, 'whole': 5.8707, 'interview': 5.8707, 'play': 5.8707, '🇺🇸': 5.8707, 'biden': 5.8707, 'harri': 5.8707, 'turn': 5.8707, 'beat': 5.8707, 'around': 5.8707, 'httpstco6j3zeyrdhm': 5.8707, 'realdonaldtrump': 5.8707, 'much': 5.8707, 'win': 5.8707, 'httpstco4fgptpb3pc': 5.8707, 'jonahdispatch': 5.8707, 'corrupt': 5.8707, 'get': 5.8707, 'cover': 5.8707, 'aren’t': 5.8707, 'least': 5.8707, 'bit': 5.8707, 'curiou': 5.8707, 'surpass': 5.8707, 'ob…': 5.8707, 'httpstco3i3svkbfe8': 5.8707, '🚨🚨break': 5.8707, 'wisconsin': 5.8707, 'presidenti': 5.8707, 'recount': 5.8707, 'complet': 5.8707, 'joe': 5.8707, 'gain': 5.8707, '87': 5.8707, 'vote': 5.8707, 'state': 5.8707, 'tru…': 5.8707, 'httpstcob0y6piut4': 5.8707, 'netflix': 5.8707, 'n

In [25]:
def get_tokens(docs, preprocess=preprocess_string, verbose=10000):
    
    for i, doc in enumerate(docs):
        yield preprocess(doc) # preprocess
        
        # print progress if needed
        if verbose > 0 and (i + 1) % verbose == 0:
            print(f"Progress: {i + 1}")
            
def get_sentences(docs, verbose=10000):
    #loop over all docs (tweets in our case)
    for i, doc in enumerate(docs):
        
        # use nltk.sent_tokenize to split paragraphs into sentences
        for sentence in nltk.sent_tokenize(doc):
            # preprocess each sentence using gensim (return string not list)
            yield " ".join(preprocess_string(sentence))
            
        # print progress if needed
        if verbose > 0 and (i + 1) % verbose == 0:
            print(f"Progress: {i + 1}")


In [26]:
# Get all tweet TEXTS
tweets, tweetsID = [], []
for t in cleanTweets:
    tweets.append(cleanTweets[t]['text']) #add tweet to list of all tweets
    tweetsID.append(cleanTweets[t]['tweetID']) #add tweet id to list of all tweet ids
    
tokens = []
for t in cleanTweets:
    tokens.append(cleanTweets[t]['tokens'])
    
sentences = list(get_sentences(tweets))
print(sentences)

['gop amp gopchairwoman immedi action expos fraud amp correct elect republican parti wil… http dwbljkdx', 'interview plai plai', 'biden amp harri turn beat http jzeyrdhm', 'realdonaldtrump win biden', 'http fgptpbpc', 'jonahdispatch corrupt get cover', 'aren’t bit curiou biden surpass ob… http isvkbf', '🚨🚨break wisconsin presidenti recount complet', 'joe biden gain vote won state', 'tru… http bypiut', 'turn netflix nfl fox new tell joe biden presid elect', 'ask', 'read… http hbnwtzyi', 'go tell blow mind presid trump won landslid victori', 'trump whine maria bartiromo “massiv dumps” sai cost elect http oxrppowbi', 'reiter fact joe biden got destroi time prior drag acro… http lfakgoahm', 'fbi thejusticedept stand america', 'oath demand', 'realdonaldtrump won', 'fight amp', 'it’ project trump', 'realdonaldtrump http gufodut', 'teddypersian realdonaldtrump thank lord scientist work vaccin doctor treatment th… http cwfhftdxy', 'end better watch media lose mind trump comeback', 'problem chr

In [27]:
#split each sentence into a list od words
words = [s.split() for s in sentences ]
for x in words:
    if "netflix" in x:
        print(x)
    

['turn', 'netflix', 'nfl', 'fox', 'new', 'tell', 'joe', 'biden', 'presid', 'elect']


In [28]:
#create a word2Vec model  
w2v_model = Word2Vec(sentences = words, size=1000, window=10, min_count=1, negative=15, sg = 1)

In [29]:
query = 'biden'
w2v_model.most_similar(query)

<ipython-input-29-0760496926b2>:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  w2v_model.most_similar(query)


[('presidenti', 0.9999772906303406),
 ('ad', 0.9999769926071167),
 ('http', 0.999976634979248),
 ('total', 0.999976634979248),
 ('break', 0.999976634979248),
 ('happen', 0.9999764561653137),
 ('campaign', 0.999976396560669),
 ('won', 0.9999761581420898),
 ('know', 0.9999760985374451),
 ('joe', 0.9999759793281555)]

In [30]:
def expand_query(query, wv, topn=10):
    
    query = preprocess_string(query)
    expanded_query = [t for t in query] # initialize with original query. Note, it is a list
    
    # extend each single term of the original query and append to expanded query
    for t in query:
        expanded_query.extend(s for s, f in wv.most_similar(t, topn=topn))
        
    return expanded_query

In [31]:
def get_top_n(bm25, query, n=10):
    #apply preprocessing to the query using get_tokens and tranform it from string to list of terms
    query = query.split() # cast query from string to list
    query = list(get_tokens(query)) # apply preprocessing
    
    query = [item for sublist in query for item in sublist] # transform list of list to list
    
    # score docs using a specific function of bm25        
    scores = np.array(bm25.get_scores(query))
    n=len(scores)
        
    # get indices of top N scores
    idx = np.argpartition(scores, -n)[-n:] # INDEX DELS TOP 20
    
    # sort top N scores and return their indices
    # if all the scores are 0 return empty list
    if np.sum(scores[idx]) == 0: 
        return[] 
    return idx[np.argsort(-scores[idx])], scores[idx[np.argsort(-scores[idx])]]

In [81]:
# Play with some queries - insert also query with typos
bm25 = BM25(tokens) # constructing a paragraph search index
expanded_query = ''
query = input()
try:
    expanded_query = ' '.join(expand_query(query, w2v_model))
except Exception as e:
    print("asdasd")
    print(e)

top_idx = None
print('Original query: {}'.format(query))
print('Expanded query: {}'.format(expanded_query))
print('---')
try:
    indexTmp, score = get_top_n(bm25, expanded_query, n=20)
    data = {'id':  [], 'text': [], 'scoreW2V': [], 'score2': []}
    df = pd.DataFrame (data, columns = ['id','text', 'scoreW2V', 'score2', 'totalScore'])

    for i in zip(indexTmp,score):
        idx=i[0]
        score=i[1]
        df = df.append({'id': tweetsID[idx], 'text': tweets[idx], 'scoreW2V': score}, ignore_index=True)
        
    ranked_docs = search_tf_idf(query, index) 
    
    for score, idx in ranked_docs:
        df['score2'][df['id'] == idx] = score  
        
    df = df.fillna(0)
    df['totalScore'] = df['scoreW2V'] + df['score2']
    df = df.sort_values(by=['totalScore'], ascending=False)
    dfList=df.values.tolist()
    for x in dfList[:20]:
        print('\nTweet ID: {}'.format(x[0]))
        print('\nTweet: {}'.format(x[1]))
        print('\nScore: {}\n'.format(x[4]))
        print('*****************************')
except:
    print("No matching documents found")

joe biden


<ipython-input-30-84dae102cdd8>:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  expanded_query.extend(s for s, f in wv.most_similar(t, topn=topn))


Original query: joe biden
Expanded query: joe biden make said love got lose ad democrat evid twitter have presidenti ad http total break happen campaign won know joe
---

Tweet ID: 1333092485577498633

Tweet: Trump is losing Twitter followers and I LOVE IT!

Score: 17.031018601440582
*****************************

Tweet ID: 1333092478493323264

Tweet: Breaking News: Japan has declared Democrat Joe Biden the winner of the election. #Fresh

Score: 15.185121193904685
*****************************

Tweet ID: 1333092457685291010

Tweet: @EricTrump Joe Biden got only 30 million votes from radical democrats, ANTIFA, BLM.

Score: 15.025573136711172
*****************************

Tweet ID: 1333029019722375180

Tweet: BREAKING: Joe Biden won absolutely NOTHING!!!!!!

Score: 13.920741713242863
*****************************

Tweet ID: 1332864570176663552

Tweet: BREAKING: Evidence Suggests Biden And Kamala Committed Campaign Finance Fraud… This Is BIG https://t.co/x7riMrcs6d

Score: 13.56853944565

<ipython-input-81-1a020e7c9475>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score2'][df['id'] == idx] = score


In [74]:
df.head(20))

,id,text,scoreW2V,score2,totalScore
0,1333092485577498633,Trump is losing Twitter followers and I LOVE IT!,17.031019,0.000000,17.031019
1,1333092478493323264,Breaking News: Japan has declared Democrat Joe...,14.585121,0.600000,15.185121
2,1333092457685291010,@EricTrump Joe Biden got only 30 million votes...,14.425573,0.600000,15.025573
3,1333029019722375180,BREAKING: Joe Biden won absolutely NOTHING!!!!!!,13.289074,0.631668,13.920742
4,1332864570176663552,BREAKING: Evidence Suggests Biden And Kamala C...,13.143169,0.425370,13.568539
5,1333089405523824644,BREAKING: Wisconsin's presidential recount is ...,12.582664,0.604661,13.187325
6,1333090021134438401,🚨WAR ROOM ALERT: The presidential recount in W...,11.459780,0.601839,12.061618
7,1333075330584743936,President @realDonaldTrump: There is no way Jo...,10.854807,0.649242,11.504049
8,1333092446834716673,"@realDonaldTrump ""... I can't believe Joe Bide...",10.400035,0.600000,11.000035
10,1332876807138373635,Do you really believe that Joe Biden elicited ...,10.077344,0.622068,10.699412
